<a href="https://colab.research.google.com/github/LeeSeungYun1020/Machine_Learning/blob/main/PyTorch_Tutorial/06_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 파라미터 최적화

## 기본 코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 하이퍼파라미터

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 최적화 루프
에포크: 최적화 단계의 각 반복(이터레이션)
- 학습
- 검증

## 손실 함수
예측과 정답을 비교하여 손실 계산
- MSE (평균 제곱 오차)
- NLL (음의 로그 가능도)
- CE (크로스 엔트로피, LogSoftmax, NLLLoss 합침)

In [4]:
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

- optimizer.zero_grad()  
모델 매개변수 변화도 재설정
- loss.backward()  
예측 손실을 역전파, 손실의 변화도 저장
- optimizer.step()  
역전파 단계의 변화도로 매개변수 조정

## 전체 구현

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299600  [    0/60000]
loss: 2.296324  [ 6400/60000]
loss: 2.284041  [12800/60000]
loss: 2.276942  [19200/60000]
loss: 2.274832  [25600/60000]
loss: 2.245072  [32000/60000]
loss: 2.256315  [38400/60000]
loss: 2.235695  [44800/60000]
loss: 2.221474  [51200/60000]
loss: 2.212642  [57600/60000]
Test Error: 
 Accuracy: 45.0%, Avg loss: 2.212956 

Epoch 2
-------------------------------
loss: 2.219324  [    0/60000]
loss: 2.227234  [ 6400/60000]
loss: 2.193108  [12800/60000]
loss: 2.186720  [19200/60000]
loss: 2.198094  [25600/60000]
loss: 2.136287  [32000/60000]
loss: 2.156644  [38400/60000]
loss: 2.116619  [44800/60000]
loss: 2.085719  [51200/60000]
loss: 2.081478  [57600/60000]
Test Error: 
 Accuracy: 54.0%, Avg loss: 2.081084 

Epoch 3
-------------------------------
loss: 2.100972  [    0/60000]
loss: 2.114411  [ 6400/60000]
loss: 2.044212  [12800/60000]
loss: 2.032355  [19200/60000]
loss: 2.065195  [25600/60000]
loss: 1.969747  [32000/600